<a href="https://colab.research.google.com/github/preetilakade/AIMathAssistantUsingLangGraph/blob/main/AImathAssistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ai math Assistant using langchain


In [1]:
%pip install langchain==0.3.23
%pip install langchain-ibm==0.3.10
%pip install langchain-community==0.3.16
%pip install wikipedia==1.4.0
%pip install openai==1.77.0
%pip install langchain-openai==0.3.16



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 458.9/458.9 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 91.1 MB/s eta 0:00:00
  Attempting uninstall: zstandard
    Found existing installation: zstandard 0.25.0
    Uninstalling zstandard-0.25.0:
      Successfully uninstalled zstandard-0.25.0
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.6.4
    Uninstalling langsmith-0.6.4:
      Successfully uninstalled langsmith-0.6.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.7
    Uninstalling langchain-core-1.2.7:
      Successfully uninstalled langchain-core-1.2.7
  Attempting uninstall: langchain
    Found existing installation: langchain 1.2.4
    Uninstalling langchain-1.2.4:
      Successfully uninstalled langchain-1.2.4
ERROR

required libraries

In [2]:
from langchain_ibm import ChatWatsonx
from langchain.agents import AgentType
import re

for this project we use chatwatsonX LLM

In [4]:
from langchain_openai import ChatOpenAI
from langchain_ibm import ChatWatsonx

openai_llm = ChatOpenAI(
    model="gpt-4.1-nano",
    api_key = "your openai api key here",
)

watsonx_llm = ChatWatsonx(
    model_id="ibm/granite-3-2-8b-instruct",
    url="https://us-south.ml.cloud.ibm.com",
    project_id="your project id associated with the API key",
    api_key="your watsonx.ai api key here",
)

ValidationError: 1 validation error for ChatWatsonx
  Value error, Did not find 'apikey' or 'token', please add an environment variable `WATSONX_APIKEY` or 'WATSONX_TOKEN' which contains it, or pass 'apikey' or 'token' as a named parameter. [type=value_error, input_value={'model_id': 'ibm/granite...atsonx.ai api key here'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error

In [ ]:
def add_numbers(inputs:str) -> dict:
    """
    Adds a list of numbers provided in the input dictionary or extracts numbers from a string.

    Parameters:
    - inputs (str):
    string, it should contain numbers that can be extracted and summed.

    Returns:
    - dict: A dictionary with a single key "result" containing the sum of the numbers.

    Example Input (Dictionary):
    {"numbers": [10, 20, 30]}

    Example Input (String):
    "Add the numbers 10, 20, and 30."

    Example Output:
    {"result": 60}
    """
    numbers = [int(x) for x in inputs.replace(",", "").split() if x.isdigit()]


    result = sum(numbers)
     return {"result": result}

In [ ]:
add_numbers("1 2")

In [ ]:
from langchain.agents import Tool
add_tool=Tool(
        name="AddTool",
        func=add_numbers,
        description="Adds a list of numbers and returns the result.")

print("tool object",add_tool)

In [ ]:
print("Tool Name:")
print(add_tool.name)

# Tool description
print("Tool Description:")
print(add_tool.description)

# Tool function
print("Tool Function:")
print(add_tool.invoke)

In [ ]:
print("Calling Tool Function:")
test_input = "10 20 30 a b"
print(add_tool.invoke(test_input))

addition with two parameters:
absolute:false- simply adds numbers
absolute:true - turns ne-ve nums to p+ve ans then adds

In [ ]:
from typing import List

@tool
def add_numbers_with_options(numbers: List[float], absolute: bool = False) -> float:
    """
    Adds a list of numbers provided as input.

    Parameters:
    - numbers (List[float]): A list of numbers to be summed.
    - absolute (bool): If True, use the absolute values of the numbers before summing.

    Returns:
    - float: The total sum of the numbers.
    """
    if absolute:
        numbers = [abs(n) for n in numbers]
    return sum(numbers)

In [ ]:
print(add_numbers_with_options.invoke({"numbers":[-1.1,-2.1,-3.0],"absolute":False}))
print(add_numbers_with_options.invoke({"numbers":[-1.1,-2.1,-3.0],"absolute":True}))

sum_numbers_with_complex_output formats structure and provides detailed output

In [ ]:
from typing import Dict, Union

@tool
def sum_numbers_with_complex_output(inputs: str) -> Dict[str, Union[float, str]]:
    """
    Extracts and sums all integers and decimal numbers from the input string.

    Parameters:
    - inputs (str): A string that may contain numeric values.

    Returns:
    - dict: A dictionary with the key "result". If numbers are found, the value is their sum (float).
            If no numbers are found or an error occurs, the value is a corresponding message (str).

    Example Input:
    "Add 10, 20.5, and -3."

    Example Output:
    {"result": 27.5}
    """
    matches = re.findall(r'-?\d+(?:\.\d+)?', inputs)
    if not matches:
        return {"result": "No numbers found in input."}
    try:
      numbers = [float(num) for num in matches]
        total = sum(numbers)
        return {"result": total}
    except Exception as e:
        return {"result": f"Error during summation: {str(e)}"}

Relationship between agent and LLM

The agent acts as the decision-maker, figuring out which tools to use and when.
The LLM is the reasoning engine. It:
Interprets the user's input.
Helps the agent make decisions.
Generates a response based on the output of the tools.

Specifies the reasoning framework for the agent.
"zero-shot-react-description" enables:
Zero-shot reasoning: The agent can solve tasks it hasn't seen before by thinking through the problem step by step.
React framework: A logical loop of:

create an agent object using initialize_agent.

In [ ]:
from langchain.agents import initialize_agent

agent = initialize_agent([add_tool], llm, agent="zero-shot-react-description", verbose=True, handle_parsing_errors=True)

In [ ]:
esponse =agent.run("In 2023, the US GDP was approximately $27.72 trillion, while Canada's was around $2.14 trillion and Mexico's was about $1.79 trillion what is the total.")

In [ ]:
agent.invoke({"input": "Add 10, 20, two and 30"})

In the agent example below, use sum_numbers_from_text as a tool and structured-chat-zero-shot-react-description as the agent type. For the LLM, you'll use Granite.

In [ ]:
agent_2 = initialize_agent([sum_numbers_from_text], llm, agent="structured-chat-zero-shot-react-description", verbose=True, handle_parsing_errors=True)
response = agent_2.invoke({"input": "Add 10, 20 and 30"})
print(response)

In [ ]:
from langchain_openai import ChatOpenAI

llm_ai = ChatOpenAI(model="gpt-4.1-nano")

In [ ]:
agent_3 = initialize_agent([sum_numbers_with_complex_output], llm_ai, agent="openai-functions", verbose=True, handle_parsing_errors=True)
response = agent_3.invoke({"input": "Add 10, 20 and 30"})
print(response)

What is create_react_agent?
It builds a graph-based ReAct agent that:
Thinks step-by-step (Reason + Act)
Calls tools
Maintains state
Supports memory
Handles multi-step reasoning

In [ ]:
%pip install langgraph==0.6.1 | tail -n 1

In [ ]:
from langgraph.prebuilt import create_react_agent

agent_exec = create_react_agent(model=llm_ai, tools=[sum_numbers_from_text])
msgs = agent_exec.invoke({"messages": [("human", "Add the numbers -10, -20, -30")]})

creating substraction tool:

In [ ]:
@tool
def subtract_numbers(inputs: str) -> dict:
    """
    Extracts numbers from a string, negates the first number, and successively subtracts
    the remaining numbers in the list.

    This function is designed to handle input in string format, where numbers are separated
    by spaces, commas, or other delimiters. It parses the string, extracts valid numeric values,
    and performs a step-by-step subtraction operation starting with the first number negated.

    Parameters:
    - inputs (str):
      A string containing numbers to subtract. The string may include spaces, commas, or
      other delimiters between the numbers.

    Returns:
    - dict:
      A dictionary containing the key "result" with the calculated difference as its value.
      If no valid numbers are found in the input string, the result defaults to 0.

    Example Input:
    "100, 20, 10"
Example Output:
    {"result": -130}

    Notes:
    - Non-numeric characters in the input are ignored.
    - If the input string contains only one valid number, the result will be that number negated.
    - Handles a variety of delimiters (e.g., spaces, commas) but does not validate input formats
      beyond extracting numeric values.
    """
    # Extract numbers from the string
    numbers = [int(num) for num in inputs.replace(",", "").split() if num.isdigit()]

    # If no numbers are found, return 0
    if not numbers:
        return {"result": 0}

    # Start with the first number negated
    result = -1 * numbers[0]

    # Subtract all subsequent numbers
    for num in numbers[1:]:
        result -= num
        return {"result": result}

lets test:

In [ ]:
print("Calling Tool Function:")
test_input = "10 20 30 and four a b"
print(subtract_numbers.invoke(test_input))

multiplication tool:

In [ ]:
@tool
def multiply_numbers(inputs: str) -> dict:
    """
    Extracts numbers from a string and calculates their product.

    Parameters:
    - inputs (str): A string containing numbers separated by spaces, commas, or other delimiters.

    Returns:
    - dict: A dictionary with the key "result" containing the product of the numbers.

    Example Input:
    "2, 3, 4"

    Example Output:
    {"result": 24}

    Notes:
    - If no numbers are found, the result defaults to 1 (neutral element for multiplication).
    """
    # Extract numbers from the string
    numbers = [int(num) for num in inputs.replace(",", "").split() if num.isdigit()]
    print(numbers)

    # If no numbers are found, return 1
    if not numbers:
        return {"result": 1}

    # Calculate the product of the numbers
    result = 1
    for num in numbers:
        result *= num
        print(num)

    return {"result": result}

division tool:

In [ ]:
@tool
def divide_numbers(inputs: str) -> dict:
    """
    Extracts numbers from a string and calculates the result of dividing the first number
    by the subsequent numbers in sequence.

    Parameters:
    - inputs (str): A string containing numbers separated by spaces, commas, or other delimiters.

    Returns:
    - dict: A dictionary with the key "result" containing the quotient.

    Example Input:
    "100, 5, 2"

    Example Output:
    {"result": 10.0}

    Notes:
    - If no numbers are found, the result defaults to 0.
    - Division by zero will raise an error.
    """
    # Extract numbers from the string
    numbers = [int(num) for num in inputs.replace(",", "").split() if num.isdigit()]


    # If no numbers are found, return 0
    if not numbers:
        return {"result": 0}

    # Calculate the result of dividing the first number by subsequent numbers
    result = numbers[0]
    for num in numbers[1:]:
        result /= num

    return {"result": result}

Building the agent:

In [ ]:
tools = [add_numbers,subtract_numbers, multiply_numbers, divide_numbers]
tools

create the agent with the tools and the language model as input.

In [ ]:
from langgraph.prebuilt import create_react_agent

# Create the agent with all tools
math_agent = create_react_agent(
    model=llm_ai,
    tools=tools,
    # Optional: Add a system message to guide the agent's behavior
    prompt="You are a helpful mathematical assistant that can perform various operations. Use the tools precisely and explain your reasoning clearly."
)

testing by invoking

In [ ]:
response = math_agent.invoke({
    "messages": [("human", "What is 25 divided by 4?")]
})

# Get the final answer
final_answer = response["messages"][-1].content
print(final_answer)

In [ ]:
print("\n--- Testing MultiplyTool ---")
response = math_agent.invoke({
    "messages": [("human", "Multiply 2, 3, and four.")]
})
print("Agent Response:", response["messages"][-1].content)

print("\n--- Testing DivideTool ---")
response = math_agent.invoke({
    "messages": [("human", "Divide 100 by 5 and then by 2.")]
})
print("Agent Response:", response["messages"][-1].content)